In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
import re
import json
import os
from process import *

if os.path.exists('data/acled_preprocessed.csv'):
    print("Preprocessed dataset found. Loading...")
    df = pd.read_csv('data/acled_preprocessed.csv')
else:
    df = pd.read_csv('data/acled.csv')
    df['event_date'] = pd.to_datetime(df['event_date'])
    df = process()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [30]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split

In [31]:
X = np.array(df.drop(columns='future_fatalities'))
y = np.array(df['future_fatalities'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# use a MLPClassifier to predict if fatalities_sum is 0 or not
clf = MLPClassifier()
clf.fit(X_train, y_train == 0)
print("Accuracy of predicting if fatalities_sum is 0:", clf.score(X_test, y_test == 0))

Accuracy of predicting if fatalities_sum is 0: 0.9245699001685577


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Instantiate the classifier with fixed parameters
clf = MLPClassifier(
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    max_iter=100,
    early_stopping=True,
    random_state=42
)

# Define the parameter grid
param_grid = {
    'learning_rate_init': [1e-4, 1e-3, 5e-4]  # Only tune learning rate
}

# Use GridSearchCV to find the best parameters
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    cv=2,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X_train, y_train == 0)

# Output accuracy for each model
results = grid_search.cv_results_
for i in range(len(results['mean_test_score'])):
    print(f"Model {i + 1}:")
    print(f"  Parameters: {results['params'][i]}")
    print(f"  Mean Validation Accuracy: {results['mean_test_score'][i]:.4f}")
    print(f"  Std Dev Validation Accuracy: {results['std_test_score'][i]:.4f}")
    print()

# Print the best parameters and accuracy
print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Evaluate on the test set
best_clf = grid_search.best_estimator_
test_accuracy = accuracy_score(y_test == 0, best_clf.predict(X_test))
print("Test set accuracy:", test_accuracy)